In [14]:
pip install --upgrade scikit-learn


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/88/a3/023bf03411b8d89954d028dd56dc97f5fa04f18b4cb2acf4315dfb3dfa15/scikit_learn-1.3.1-cp39-cp39-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 50.2 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.0.4 requires pandas<2.0.0,>=1.3.0, but you have pandas 2.1.0 which is incompatible.
pycaret 3.0.4 requires scikit-learn<1.3.0,>=1.0, but you have scikit-learn 1.3.1 which is incompatible.
sktime 0.22.0 requires pandas<2.1.0,>=1.1.0, but you have pandas 2.1.0 which is incompatible.
Note: you may need to restart the ke

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np

# Load the data
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

# One-hot encoding
train_encoded = pd.get_dummies(train_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])
X = train_encoded.drop(columns=["ID", "수요량"])
y = train_encoded["수요량"]

# Splitting data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standard scaling
scaler = StandardScaler()
X_train_standard = scaler.fit_transform(X_train)
X_val_standard = scaler.transform(X_val)

# GridSearchCV for Random Forest
rf = RandomForestRegressor()
rf_param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}
rf_search = GridSearchCV(rf, rf_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
rf_search.fit(X_train_standard, y_train)
rf_best = rf_search.best_estimator_
rf_val_predictions = rf_best.predict(X_val_standard)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_val_predictions))
print(f"Random Forest RMSE: {rf_rmse}")

# GridSearchCV for XGBoost
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'n_estimators': [100, 200, 300]
}

xgb_search = GridSearchCV(xgb_model, xgb_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_standard, y_train)
xgb_best = xgb_search.best_estimator_
xgb_val_predictions = xgb_best.predict(X_val_standard)
xgb_rmse = np.sqrt(mean_squared_error(y_val, xgb_val_predictions))
print(f"XGBoost RMSE: {xgb_rmse}")

# Preprocessing test data
test_encoded = pd.get_dummies(test_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])
X_test = test_encoded.drop(columns=["ID"])
X_test_standard = scaler.transform(X_test)

# Predicting on test data
rf_test_predictions = rf_best.predict(X_test_standard)
xgb_test_predictions = xgb_best.predict(X_test_standard)

# Creating submission files
submission_rf = pd.DataFrame({'ID': test_data["ID"], '수요량': rf_test_predictions})
submission_xgb = pd.DataFrame({'ID': test_data["ID"], '수요량': xgb_test_predictions})
submission_rf.to_csv("./data/submission_rf.csv", index=False)
submission_xgb.to_csv("./data/submission_xgb.csv", index=False)


Random Forest RMSE: 183.89063479699706
XGBoost RMSE: 163.83979973870817


In [3]:
xgb_rmse

163.83979973870817

In [4]:
rf_rmse

183.89063479699706

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Define hyperparameters grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor()

# Random search of parameters using 3 fold cross validation
rf_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_grid, 
                               n_iter=10, scoring='neg_mean_squared_error', 
                               cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_search.fit(X_train, y_train)

# Best hyperparameters from random search
best_rf = rf_search.best_estimator_

# Predict using the best model
y_pred_rf = best_rf.predict(X_val)

# Calculate RMSE for Random Forest
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version.

In [16]:
rmse_rf

184.9755360165115

In [17]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

# Define hyperparameters grid for XGBoost
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'n_estimators': [100, 200, 300]
}

# Initialize XGBoost Regressor
xgb_model = xgb.XGBRegressor()

# Random search of parameters using 3 fold cross validation
xgb_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgb_param_grid, 
                                n_iter=10, scoring='neg_mean_squared_error', 
                                cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
xgb_search.fit(X_train, y_train)

# Best hyperparameters from random search
best_xgb = xgb_search.best_estimator_

# Predict using the best model
y_pred_xgb = best_xgb.predict(X_val)

# Calculate RMSE for XGBoost
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/syshin/miniforge3/env

In [18]:
rmse_xgb

167.38069494865007

[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   3.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=5, n_estimators=150; total time=   5.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.3s
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=150; total time=   3.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   2.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=5, n_estimators=150; total time=   5.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.9s
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=150; total time=   3.4s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=